In [1]:
import os
from indexer import clone_repo, build_index, query_index

# Clone the repository if not present
repo_url = 'https://github.com/viarotel-org/escrcpy'
repo_folder = '../data/escrcpy'
if not os.path.exists(repo_folder):
    clone_repo(repo_url, repo_folder)

In [2]:
from evaluate import load_evaluation_data, evaluate

SyntaxError: invalid syntax (evaluate.py, line 21)

In [ ]:
eval_filepath = '../data/escrcpy-commits-generated.json'  # updated file path
if os.path.exists(eval_filepath):
    data = load_evaluation_data(eval_filepath)
    evaluate(repo_folder, data)
else:
    print(f'No evaluation data file found at {eval_filepath}.')

Found 346 candidate files.


No sentence-transformers model found with name microsoft/graphcodebert-base. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Recall@10: 0.00


In [ ]:
for item in data:
    # Updated key access: use 'question' and 'files'
    query = item.get('question')
    expected_files = set(item.get('files', []))

    if query is None:
        print('Skipping item without question:', item)
        continue